# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *
# try github 

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
ranker_train_data_dir = '../data/rank_train_data_v2/'
rank_model_version = 'rank_lgbm_v4'

In [3]:
debug = False

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [4]:
rank_model_dir

'../model_training/rank_lgbm_v4'

In [5]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/rank_lgbm_v4’: File exists


# Read data

In [33]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, f'sessions_test_{task}.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))
test4task3_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))



In [34]:
! ls {ranker_train_data_dir}

eval.parquet  test.parquet  test4task3.parquet	test_phase1.parquet  train


In [35]:
train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' '*.parquet'), n_rows=SAMPLE_NUM)
# train_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'train/' 'part_5.parquet'), n_rows=SAMPLE_NUM)

eval_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'), n_rows=SAMPLE_NUM)
test_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'), n_rows=SAMPLE_NUM)


In [36]:
test4task3_candidates = pl.scan_parquet(os.path.join(ranker_train_data_dir, 'test4task3.parquet'), n_rows=SAMPLE_NUM)

In [37]:
# train_candidates.collect().shape

In [38]:
! ls {ranker_train_data_dir}/train

part_1.parquet	part_2.parquet	part_3.parquet	part_4.parquet	part_5.parquet


# Ranker 

## Config 

In [23]:
estimator = 1000
early_stop_rounds = 10
eval_frac = 0.1
if debug:
    early_stop_rounds = 5
    
target = 'target'
feature_cols = [
    'next_item_weight'
    , 'next_item_weight_nfi'
    , 'next_item_weight_co_visit'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
    # , 'prev_item_similarity'	
    , 'prev_length'
    , 'price_mean_norm'	
    , 'price_last_norm'
    , 'price_min_norm'
    , 'price_max_norm'
]

categorical_feature=['locale']

## Train model

In [12]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="gbdt",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[3]
)

### eval data for training 

In [13]:
# eval_candidates.collect().shape

In [14]:
# train_candidates.collect().shape

In [15]:
# eval_candidates.head(3).collect()

In [16]:
# train_candidates.head(3).collect()

In [17]:
train_data_source = 'train' # train or 'eval'

if train_data_source == 'eval':
    target_df = eval_candidates
    negative_sampling = 0.1

    cols = ['session_id', 'next_item', 'next_item_prediction', 'target']
    sample_pl = target_df.select(cols)
    pos = sample_pl.filter(pl.col('target')==1)
    neg = (sample_pl
               .filter(pl.col('target')==0)
          )
    neg_original_len = neg.select('session_id').collect().shape[0]
    print(f'Original negative num: {neg_original_len}')
    neg = neg.groupby('session_id').agg(
        pl.all().sample(frac=negative_sampling)
        )
    neg = neg.explode(neg.columns[1:])
    # # print(neg.shape)
    # # print(neg.head().collect())
    sample_pl = pl.concat([pos, neg], how='vertical')
    target_df = sample_pl.join(target_df, how='left', on=cols).collect().sort('session_id').lazy()
    del sample_pl
else:
    target_df = train_candidates
    del train_candidates

### Train 

In [18]:
# target_df.schema

In [19]:
# target_df.select('session_id').head().collect()#.unique().collect()

In [20]:
target_train_candidates = target_df
del target_df

#### More featurs 

In [14]:
session_features = pl.scan_parquet('../data/session_item_features/session_features.parquet')
product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))

In [15]:
# session_features.schema

In [16]:
# target_train_candidates

In [17]:
# target_train_candidates.head(3).collect()

In [18]:
def get_additional_feature(data_pl, session_features=session_features, product_pl=product_pl):
    session_join_keys = ['session_id']
    session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
    data_pl = (
        data_pl.join(session_features.select(session_join_keys+session_fea_cols+['locale']),
                                    how='left', on=session_join_keys, suffix='_str')
            .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale_str'])
            .with_columns(
              pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
              , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
              , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
              , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
              # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
            )
            .select(
                pl.all().exclude(session_fea_cols+['locale_str', 'price'])
            )

    )
    return data_pl

In [55]:
target_train_candidates = get_additional_feature(data_pl=target_train_candidates)

In [56]:
train_session_id = target_train_candidates.select(pl.col('session_id')).unique().collect()

session_num = train_session_id.shape[0]

In [57]:
random_pl = train_session_id.with_columns(
        pl.Series(name='random', values=np.random.uniform(size=session_num))
)

In [58]:
train_candidates = target_train_candidates.join(random_pl.lazy(), how='left', on='session_id')

sampled_train_candidates = train_candidates.filter(pl.col('random')>eval_frac).sort('session_id', descending=False).collect()
sampled_eval_candidates = train_candidates.filter(pl.col('random')<=eval_frac).sort('session_id', descending=False).collect()

In [59]:
print('Train')
print(sampled_train_candidates.shape)
print(sampled_train_candidates.select('target').mean())
print()
print()
print('Eval')
print(sampled_eval_candidates.shape)
print(sampled_eval_candidates.select('target').mean())

Train
(13120484, 16)
shape: (1, 1)
┌─────────┐
│ target  │
│ ---     │
│ f64     │
╞═════════╡
│ 0.05024 │
└─────────┘


Eval
(1464944, 16)
shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.050214 │
└──────────┘


In [60]:
# sampled_train_candidates.head()

<!-- no positive items ->  -->

In [61]:
# sampled_train_candidates.filter(pl.col('session_id')==0).sort('target', descending=True)

In [62]:
# sampled_train_candidates.filter(pl.col('session_id')==5).sort('target', descending=True)

In [63]:
# sampled_train_candidates.head()

In [64]:
# sampled_train_candidates.groupby('session_id').agg(pl.count())

In [65]:
session_lengths_train = sampled_train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = sampled_eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [66]:
pd.DataFrame(session_lengths_train).describe([0.05*i for i in range(20)])

,0
count,659175.000000
mean,19.904402
std,9.618026
min,8.000000
0%,8.000000
5%,11.000000
10%,11.000000
15%,11.000000
20%,12.000000
25%,12.000000


In [67]:
pd.DataFrame(session_lengths_eval).describe([0.05*i for i in range(20)])

,0
count,73561.000000
mean,19.914683
std,9.642499
min,9.000000
0%,9.000000
5%,11.000000
10%,11.000000
15%,11.000000
20%,12.000000
25%,12.000000


In [68]:
ranker = ranker.fit(
    X=sampled_train_candidates[feature_cols].to_pandas(),
    y=sampled_train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(sampled_train_candidates[feature_cols].to_pandas(), sampled_train_candidates[target].to_pandas()),
           (sampled_eval_candidates[feature_cols].to_pandas(), sampled_eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    , early_stopping_rounds=early_stop_rounds
    # , callbacks=[early_stopping(early_stop_rounds)]
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib

[1]	valid_0's ndcg@3: 0.884845	valid_1's ndcg@3: 0.885292
[2]	valid_0's ndcg@3: 0.8734	valid_1's ndcg@3: 0.873061
[3]	valid_0's ndcg@3: 0.872069	valid_1's ndcg@3: 0.872139
[4]	valid_0's ndcg@3: 0.871209	valid_1's ndcg@3: 0.871253
[5]	valid_0's ndcg@3: 0.870109	valid_1's ndcg@3: 0.870364
[6]	valid_0's ndcg@3: 0.869718	valid_1's ndcg@3: 0.869742
[7]	valid_0's ndcg@3: 0.867055	valid_1's ndcg@3: 0.866963
[8]	valid_0's ndcg@3: 0.867168	valid_1's ndcg@3: 0.867102
[9]	valid_0's ndcg@3: 0.867521	valid_1's ndcg@3: 0.867506
[10]	valid_0's ndcg@3: 0.86666	valid_1's ndcg@3: 0.866818
[11]	valid_0's ndcg@3: 0.865678	valid_1's ndcg@3: 0.866113


In [69]:
print(sampled_train_candidates.estimated_size(unit='gb'))
del sampled_train_candidates

1.5213156826794147


In [70]:
print(sampled_eval_candidates.estimated_size(unit='gb'))
del sampled_eval_candidates

0.16985977441072464


## Save model 

In [71]:
debug

False

In [72]:
if not debug:
    joblib.dump(
            value=ranker,
            filename=os.path.join(rank_model_dir, 'model.pkl')
    )

In [73]:
if not debug:
    del ranker

## Load Model 

In [19]:
debug

False

In [20]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

### Importance 

In [21]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
0,next_item_weight,3.988920e+06
7,price_mean_norm,6.407572e+04
1,next_item_weight_nfi,6.850975e+03
2,next_item_weight_co_visit,4.961293e+03
5,last_item_similarity,2.780719e+03
4,w2v_weight,1.429808e+03
6,prev_length,6.914070e+02
9,price_min_norm,5.488500e+02
10,price_max_norm,3.044230e+02
3,locale,0.000000e+00


# Eval ranker 

In [76]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [77]:
# target_train_candidates.head().collect()

In [78]:
eval_candidates = get_additional_feature(data_pl=eval_candidates)

In [79]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (64153741, 15)


In [80]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.3335,0.4702,0.5563


In [46]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

# Test inference 

In [25]:
test_candidates = get_additional_feature(data_pl=test_candidates)

In [26]:
test_candidates = test_candidates.collect()
test_candidates.shape

(74361010, 13)

In [29]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [30]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()

In [31]:
test_pl.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [32]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [33]:
predictions.shape

(316972, 2)

In [34]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

# test2task3 

In [39]:

topn = 1
test4task3_candidates = get_additional_feature(data_pl=test4task3_candidates)
test4task3_candidates = test4task3_candidates.collect()
inference = ranker.predict(test4task3_candidates[feature_cols].to_pandas())
test4task3_result = (test4task3_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=topn)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test4task3_result.head().collect()
predictions = test4task3_pl.join(test4task3_result, how='left', on='session_id').collect()
# [['locale', 'next_item_prediction']].to_pandas()

In [40]:
test4task3_sub_file = f'../data/sub_files/{task}_test4task3_{rank_model_version}.parquet'
print(test4task3_sub_file)
predictions.write_parquet(test4task3_sub_file)
predictions.select('locale').to_series().value_counts()

../data/sub_files/task1_test4task3_rank_lgbm_v4.parquet


locale,counts
str,u32
"""UK""",10000
"""DE""",10000
"""JP""",10000


# Submit result 

In [35]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}

rank_lgbm_v4.parque ━━━━━━━━━━━━━━━ 100.0% • 299.5/299.5 MB • 2.9 MB/s • 0:00:0000:0100:07
                                                                                 ╭─────────────────────────╮                                                                                 
                                                                                 │ Successfully submitted! │                                                                                 
                                                                                 ╰─────────────────────────╯                                                                                 
                                                                                       Important links                                                                                       
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────